In [ ]:
import os
import json
import pprint
import re
from PIL import Image as PILImage
from Prompts import *
import pickle as pkl
import yaml
from utils.config import *
from utils.models import *
import copy
import subprocess
model = GPTModel(config = dict(
    MODEL_NAME = MODEL
))
with open(os.path.join('locations',LOCATION,'scene_graph.json'),'r') as f:
    scene_graph = json.load(f)

scgraph = utils.SceneGraph(scene_graph)
node_types = []
for node in scgraph.get_parent_nodes():
    if scgraph.graph.nodes[node]['type'] not in node_types:
        node_types.append(scgraph.graph.nodes[node]['type'])
print(node_types)

In [ ]:
scQ = ScenarioQuery()
scQ_full_prompt = scQ.get_full_prompt(context=CONTEXT,
    task=TASK,
    rough_scenario=ROUGH_SCENARIO,
    location=LOCATION_DESC
)

In [ ]:
scQ_full_prompt

### Query for Scenario

In [ ]:
if QUERY_SC:
    response = model.get_response(messages = scQ_full_prompt,format = "json_object")
    scq_response_json = json.loads(response)
    if SAVE_SC_RESPONSE:
        with open('responses/reponse_sc.json','w') as f:
            json.dump(scq_response_json,f)
else:
    assert LOAD_SC_RESPONSE == True
    with open('responses/reponse_sc.json','r') as f:
        scq_response_json = json.load(f)
print(scq_response_json.keys())
pprint.pprint(scq_response_json)

In [ ]:
if USE_HANDCRAFTED_SCENARIO:
    scenario_desc = scenario_desc_hc
    num_humans = num_humans_hc
    traj_desc = traj_desc_hc
    behav_desc = behav_desc_hc
else:
    scenario_desc = scq_response_json['Scenario Description']
    num_humans = scq_response_json['Number of Humans']
    #traj_desc = scq_response_json['Trajectories']
    behav_desc = scq_response_json['Human Behavior']
    #human_task = scq_response_json['Human Task']
    #reaction_to_robot = scq_response_json['Reaction to Robot']

# Assume the output is formatted correctly
expected_robot_behav_desc = scq_response_json['Expected Robot Behavior'] 
#pranking_desc = scq_response_json['Principle Ranking'] 
#reasoning_desc = scq_response_json['Reasoning'] 

### Fine Locations Query

In [ ]:
#scenario_desc = """The robot starts from a narrow passageway (from Node 99) with a box to transport and needs to navigate through the warehouse. As it exits the passageway, it encounters an employee coming from the opposite end of the narrow passageway, moving directly towards the robot. The passageway leads into an open area with multiple intersections branching off into other sections of the warehouse. The robot needs to decide whether to stop or proceed based on the human's behavior."""
#num_humans = 1

In [ ]:
flocationQ= FLocationQuery(
    scene_graph_img = None
)
flq_full_text_prompt = flocationQ.get_full_prompt(
    sc_desc = scenario_desc,
    scene_graph_nodes = ','.join(node_types)+',child',
    num_humans = num_humans,
)

In [ ]:
### Use get files
sg_json_file = model.get_files_by_name('scene_graph.json')
sg_img_file = model.get_files_by_name('scene_graph.png')
if sg_json_file!=None:
    print("Loaded Json SG file")
else:
    print("Creating JSON SG file")
    sg_json_file = model.create_file(os.path.join('locations',LOCATION,'scene_graph.json'), purpose = 'assistants')

if sg_img_file!=None:
    print("Loaded Image SG file")
else:
    print("Creating Image SG file")
    sg_img_file = model.create_file(os.path.join('locations',LOCATION,'scene_graph.png'), purpose = 'vision')


In [ ]:
location_content = [
            {
             "type":"text",
             "text":flq_full_text_prompt
            },
            {
                "type":"image_file",
                "image_file":{
                    "file_id":sg_img_file.id,
                    "detail":"high"
                }
            },
        ]
location_attachments = [
             {"file_id":sg_json_file.id,"tools":[{"type":"file_search"}]}
        ]

In [ ]:
# get assistant
location_assistant = model.get_assistant_by_name('location_scenario_assistant')
if location_assistant == None:
    print("Creating New Assistant")
    #create assistant
    location_assistant = model.client.beta.assistants.create(
        instructions="You are an expert floor planner and a software engineer. When asked a question, always return an answer that is fully parseable with python json.loads",
        name="location_scenario_assistant",
        tools=[{"type": "file_search"}],
        model=model.model_name,
    )
else:
    print("Loaded Assistant")

In [ ]:
thread_id, run_id = model.create_and_run_thread(
    #content = location_content,
    content = [
      {
          "type":"text",
          "text":"""
          The image shows a location which is represented by a scene graph. A scene graph which is a graph with nodes (numbered red circles) representing locations and edges (blue lines) connecting them. 
A person/robot can only move from one node to another if the two nodes are connected by an edge. 
The scene graph in json file format is also attached, where each node has an unique id, a type, the pixel position of the node in the image (pos). 
The type of a node is one among [<NODE TYPES>]. The non-child nodes represent larger areas while the child nodes represent specific places within a larger area. 
The graph is bidirectional and each edge is also represented with the edge list 'links' in the json file with double sided arrows '<->'.  
Remember that a node can only be reached from another node if they have an edge between them in the scene graph json file.
Which nodes in the graph are of type CORNER and which nodes are each of those connected directly to?"""
      }  
    ],
    attachments = location_attachments,
    assistant_id = location_assistant.id
)

response = model.poll_run_result(thread_id,run_id)

### Get Fine Locations from LLM

In [ ]:
if QUERY_LOC:
    payload = model.get_payload(content = flq_full_prompt)
    response = model.get_response(messages = payload,format = 'json_object')
    flq_response_json = json.loads(response)
    if SAVE_LOC_RESPONSE:
        with open('responses/reponse_loc.json','w') as f:
            json.dump(flq_response_json,f)
else:
    assert LOAD_LOC_RESPONSE == True
    with open('responses/reponse_loc.json','r') as f:
        flq_response_json = json.load(f)
print(flq_response_json.keys())
pprint.pprint(flq_response_json)

In [ ]:
trajectories = {}
for k,v in flq_response_json['Trajectory'].items():
    trajectories[k] = v.split(',')
print(trajectories)

In [ ]:
# Hard coded for current small warehouse map scene
def pix2world(px):
    return [(px[0]/3.0) * 0.050000 + -7.000 ,-1*((px[1]/3.0) * 0.050000 + -10.500000)]

In [ ]:
nodes_positions = {}
for node in scene_graph['nodes']:
    nodes_positions[node['id']] = pix2world(node['pos'])

In [ ]:
trajectories_world_coords = {}
for k,v in trajectories.items():
    trajectories_world_coords[k] = []
    for loc in v:
        trajectories_world_coords[k].append(nodes_positions[loc])
print(trajectories_world_coords)

#### Add fine locations to sim yaml files

In [ ]:
agents_yaml = {'hunav_loader': {'ros__parameters': {'map': LOCATION,
   'publish_people': True,
   'agents': []}}}
blank_human = {'id': None,
    'skin': 0,
    'behavior': 0,
    'group_id': -1,
    'max_vel': 1.5,
    'radius': 0.4,
    'init_pose': {'x': None, 'y': None, 'z': 1.25, 'h': 0.0},
    'goal_radius': 0.3,
    'cyclic_goals': False,
    'goals': [],
    }
agents = {}

In [ ]:
for i in range(num_humans):
    agents_yaml['hunav_loader']['ros__parameters']['agents'].append(f'agent{i}')
    agents[f'agent{i}'] = copy.deepcopy(blank_human)
    agents[f'agent{i}']['id'] = i
    agents[f'agent{i}']['behavior'] = 7+i
    for j,g in enumerate(trajectories_world_coords[f'Human {i+1}']):
        if j == 0:
            agents[f'agent{i}']['init_pose'] = {
                'x':g[0],
                'y':g[1],
                'z':1.25,
                'h':0.0,
            } 
        else:
            agents[f'agent{i}']['goals'].append(f'g{j}')
            agents[f'agent{i}'][f'g{j}'] = {
                'x':g[0],
                'y':g[1],
                'h':1.25
            }
agents_yaml['hunav_loader']['ros__parameters'].update(agents)

In [ ]:
robot_start_world_coords = pix2world(nodes_positions[int(traj_desc['Robot'][0])])
with open(HUNAV_SIM_AGENTS_FILE,'w') as f:
    yaml.dump(agents_yaml,f)
with open(os.path.join(HUNAV_GAZEBO_WRAPPER_DIR,'config','robot.yaml'),'w') as f:
    yaml.dump({
    'x_pose': robot_start_world_coords[0],
    'y_pose': robot_start_world_coords[1]
},f)

### Query LLM for BT for humans

In [ ]:
behav_desc = {
    'Human 1':""""""
}

behav_desc['Human 1'] = 'Follows the robot from location 1 to location 2 for inventory check.'
behav_desc['Human 2'] = 'Crosses the path of the robot and Human 1 at location 3.'
behav_desc['Human 3'] = 'Moves in the same direction as the robot between location 4 and location 5 but faster, potentially overtaking.'

In [ ]:
custom_node_requests = []
for i in range(num_humans):
    btq = BTQuery()
    if QUERY_BT:
        print(behav_desc[f'Human {i+1}'])
        btq_full_prompt = btq.get_full_prompt(behavior = behav_desc[f'Human {i+1}'])
        payload = model.get_payload(content = btq_full_prompt)
        response = model.get_response(messages = payload,format = 'json_object')
        btq_response_json = json.loads(response)
        if SAVE_BT_RESPONSE:
            with open(f'responses/reponse_bt_{i+1}.json','w') as f:
                json.dump(btq_response_json,f)
    else:
        assert LOAD_BT_RESPONSE == True
        with open(f'responses/reponse_bt_{i+1}.json','r') as f:
            btq_response_json = json.load(f)
        #print(btq_response_json.keys())
    pprint.pprint(btq_response_json)
    print('----')
    bt_xml = btq_response_json['Tree']
    for k,v in btq_response_json.items():
        if 'custom' in str.lower(k):
            custom_node_requests.append(v)
    with open(os.path.join(HUNAV_SIM_BT_FOLDER,f'LLMBT_{i}.xml'),'w') as f:
        f.write(bt_xml)
    print(f"Wrote BT to LLMBT_{i}.xml")

### Query LLM for custom Nodes and Auxillary functions

In [ ]:
print(custom_node_requests)

In [ ]:
for i in range(len(custom_node_requests)):
    if QUERY_AUX:
        ctnq = NodeQuery()
        ctnq_full_prompt = ctnq.get_full_prompt(description = custom_node_requests[i])
        payload = model.get_payload(content = ctnq_full_prompt)
        response = model.get_response(messages = payload,format = 'json_object')
        ctnq_response_json = json.loads(response)
        if SAVE_AUX_RESPONSE:
            with open('responses/reponse_aux.json','w') as f:
                json.dump(ctnq_response_json,f)
    else:
        assert LOAD_AUX_RESPONSE == True
        with open('responses/reponse_aux.json','r') as f:
            ctnq_response_json = json.load(f)
    with open(os.path.join('templates','extended_bt_functions.cpp'),'r') as f:
        btf_cpp = f.read()

    with open(os.path.join('templates','extended_bt_functions.hpp'),'r') as f:
        btf_hpp = f.read()

    #register node in bt_node.cpp
    with open(os.path.join('templates','extended_agent_manager.cpp'),'r') as f:
        agm_cpp = f.read()

    with open(os.path.join('templates','extended_agent_manager.hpp'),'r') as f:
        agm_hpp = f.read()

    with open(os.path.join('templates','extended_bt_node.cpp'),'r') as f:
        btn_cpp = f.read()

    with open(os.path.join('templates','extended_bt_node.hpp'),'r') as f:
        btn_hpp = f.read()
    
    #Write functions to extended_bt_functions.cpp file
    btf = ctnq_response_json['NODE_DEFINITION']
    btf = btf.replace('BT::NodeStatus BTfunctions::','BT::NodeStatus BTfunctionsExt::')
    btf_name = ctnq_response_json['NODE_NAME']
    btf_type = ctnq_response_json['NODE_TYPE'].lower().capitalize()
    btfn_name = btf_name[0].lower() + btf_name[1:]
    btf_header = ctnq_response_json['NODE_HEADER']

    btf_cpp = btf_cpp.replace('//<NEW FUNCTION>','//<NEW FUNCTION> \n' + btf)
    btf_hpp = btf_hpp.replace('//<NEW PUBLIC FUNCTION>','//<NEW PUBLIC FUNCTION> \n' + btf_header)
    
    #register BT nodes in extended_bt_node.cpp
    #   3 ports are available for each BT node:
    #       simple_port: agent_id
    #       visibleports: agent_id + distance
    #       portsNav: agent_id + timestep
    
    node_register = f"""factory_.registerSimple{btf_type}("{ctnq_response_json['NODE_NAME']}",std::bind(&BTfunctionsExt::{btfn_name},&btfunc_, _1),PORT);"""
    if ctnq_response_json['PORTS_USED'] == ['agent_id','distance']:
        node_register = node_register.replace('PORT','visibleports')
    elif ctnq_response_json['PORTS_USED'] == ['agent_id','time_step']:
        node_register = node_register.replace('PORT','portsNav')
    elif ctnq_response_json['PORTS_USED'] == ['agent_id']:
        node_register = node_register.replace('PORT','simple_port')
    else:
        node_register = node_register.replace(',PORT','')

    btn_cpp = btn_cpp.replace('//<NEW NODE REGISTER>','//<NEW NODE REGISTER> \n' + node_register)   

    #add aux functions
    for j,agmf in enumerate(ctnq_response_json['AUX_FUNCTIONS']):
        agmf = agmf.replace('void AgentManager::','void AgentManagerExt::')
        agm_cpp = agm_cpp.replace('//<NEW FUNCTION>','//<NEW FUNCTION> \n' + agmf)
        agm_hpp = agm_hpp.replace('//<NEW PUBLIC FUNCTION>','//<NEW PUBLIC FUNCTION> \n' + ctnq_response_json['AUX_FUNCTION_HEADERS'][j])

with open(os.path.join(HUNAV_SIM_CPP_FOLDER,'extended_bt_functions.cpp'),'w') as f:
    f.writelines(btf_cpp)

with open(os.path.join(HUNAV_SIM_HPP_FOLDER,'extended_bt_functions.hpp'),'w') as f:
    f.writelines(btf_hpp)

with open(os.path.join(HUNAV_SIM_CPP_FOLDER,'extended_bt_node.cpp'),'w') as f:
    f.writelines(btn_cpp)

with open(os.path.join(HUNAV_SIM_HPP_FOLDER,'extended_bt_node.hpp'),'w') as f:
    f.writelines(btn_hpp)

with open(os.path.join(HUNAV_SIM_CPP_FOLDER,'extended_agent_manager.cpp'),'w') as f:
    f.writelines(agm_cpp)

with open(os.path.join(HUNAV_SIM_HPP_FOLDER,'extended_agent_manager.hpp'),'w') as f:
    f.writelines(agm_hpp)

### Build Project

In [ ]:
s = subprocess.getstatusoutput(f' cd ~/catkin_ws && colcon build')
if s[0] == 0:
    print('Build Successful')
else:
    print('Build Failed')
    print(s[1]) 